In [12]:
import pandas as pd

# 读取第一个CSV文件
df1 = pd.read_csv(r'.\data\year_country_sports_medals.csv')

# 读取第二个CSV文件
df2 = pd.read_csv(r'.\data\merged_final_data.csv')

# 确保列名没有空格或不可见字符
df1.columns = df1.columns.str.strip()
df2.columns = df2.columns.str.strip()

# 选择 df2 的 NOC、Year 列以及从第 8 列开始的 73 列
df2_selected = df2.iloc[:, [2, 3] + list(range(7, 80))]  # 假设 NOC 和 Year 是第 0 列和第 1 列

# 检查 df2_selected 的列名
print("df2_selected 列名:", df2_selected.columns.tolist())

# 合并数据
merged_df = pd.merge(df1, df2_selected, on=['NOC', 'Year'], how='left')

# 去除所有含空单元格的行
merged_df_cleaned = merged_df.dropna()

# 检查清理后的数据
print("清理后的数据:")
print(merged_df_cleaned.head())

# 保存清理后的数据
merged_df_cleaned.to_csv('merged_output.csv', index=False)
print("清理完成，结果已保存到 merged_output.csv")

df2_selected 列名: ['NOC', 'Year', 'AthletesCount', 'Diving', 'Marathon Swimming', 'Swimming', 'Water Polo', 'Archery', 'Athletics', 'Badminton', 'Baseball', 'Softball', '3x3', 'Basketball', 'Basque Pelota', 'Boxing', 'Breaking', 'Sprint', 'Slalom', 'Cricket', 'Croquet', 'BMX Freestyle', 'BMX Racing', 'Mountain Bike', 'Road', 'Track', 'Dressage', 'Eventing', 'Jumping', 'Vaulting', 'Driving', 'Fencing', 'Field hockey', 'Flag football', 'Football', 'Golf', 'Artistic', 'Rhythmic', 'Trampoline', 'Indoor', 'Field', 'Jeu de Paume', 'Judo', 'Karate', 'Sixes', 'Field.1', 'Modern Pentathlon', 'Polo', 'Rackets', 'Roque', 'Coastal', 'Rowing', 'Sevens', 'Union', 'Sailing', 'Shooting', 'Skateboarding', 'Sport Climbing', 'Squash', 'Surfing', 'Table Tennis', 'Taekwondo', 'Tennis', 'Triathlon', 'Tug of War', 'Beach', 'Indoor.1', 'Water Motorsports', 'Weightlifting', 'Freestyle', 'Greco-Roman', 'Total events', 'Total disciplines', 'Total sports', 'Total_Sports_Events']
清理后的数据:
           NOC  Year      S

In [17]:
import pandas as pd

# 读取两个CSV文件
country_sports_event_medals = pd.read_csv(r'.\data\merged_year_country_sports_medals.csv')
summerOly_hosts = pd.read_csv(r'.\data\summerOly_hosts.csv')

# 合并两个数据框，按照Year进行匹配
merged_df = pd.merge(country_sports_event_medals, summerOly_hosts, on='Year', how='left')

# 新增isHost列，判断Host Country是否等于Country
merged_df['isHost'] = (merged_df['Host Country'] == merged_df['NOC']).astype(int)

# 删除多余的列（如果需要）
merged_df.drop(columns=['Host Country'], inplace=True)

# 保存结果到新的CSV文件（可选）
merged_df.to_csv('updated_country_sports_event_medals.csv', index=False)

# 打印结果
print(merged_df.head())

           NOC  Year      Sport  Gold  Medals  AthletesCount  Diving  \
0  Afghanistan  2008  Taekwondo     0       1              4       8   
1  Afghanistan  2012  Taekwondo     0       1              6       8   
2      Albania  2024  Wrestling     0       2              9       8   
3      Algeria  1984     Boxing     0       2             33       4   
4      Algeria  1992  Athletics     1       1             36       4   

   Marathon Swimming  Swimming  Water Polo  ...  Water Motorsports  \
0                  2        32           2  ...                  0   
1                  2        32           2  ...                  0   
2                  2        35           2  ...                  0   
3                  0        29           1  ...                  0   
4                  0        31           1  ...                  0   

   Weightlifting  Freestyle  Greco-Roman  Total events  Total disciplines  \
0             15         11            7           302               

In [19]:
import pandas as pd

# 读取CSV文件
df = pd.read_csv(r'.\data\country_sports_event_medals.csv')

# 筛选出所有东道主国家
host_countries = df[df['isHost'] == 1]['NOC'].unique()

# 创建一个空的DataFrame来存储结果
result_df = pd.DataFrame()

# 遍历每个东道主国家
for noc in host_countries:
    # 过滤出该国家的所有数据
    country_data = df[df['NOC'] == noc]
    
    # 将过滤出的数据添加到结果DataFrame中
    result_df = pd.concat([result_df, country_data])

# 重置索引
result_df.reset_index(drop=True, inplace=True)

# 输出结果
print(result_df)

# 如果需要保存结果到新的CSV文件
result_df.to_csv(r'.\data\host_countries_medals.csv', index=False)

                NOC  Year          Sport  Gold  Medals  isHost  AthletesCount  \
0         Australia  1896      Athletics     2       2       0              5   
1         Australia  1896         Tennis     0       1       0              5   
2         Australia  1900      Athletics     1       4       0              6   
3         Australia  1900       Swimming     2       2       0              6   
4         Australia  1904       Swimming     0       4       0              6   
...             ...   ...            ...   ...     ...     ...            ...   
3171  United States  2024         Tennis     0       4       0            854   
3172  United States  2024     Volleyball     0      26       0            854   
3173  United States  2024     Water Polo     0      13       0            854   
3174  United States  2024  Weightlifting     1       2       0            854   
3175  United States  2024      Wrestling     2       7       0            854   

      Diving  Marathon Swim

In [34]:
import pandas as pd

# 读取数据
df = pd.read_csv(r'.\data\host_countries_medals.csv')

# 打印前几行数据，确认数据是否正确加载
print("原始数据前几行：")
print(df.head())

# 使用 pivot 将 Sport 列转换为新的列
df_pivot = df.pivot_table(index=['NOC', 'Year'], columns='Sport', values=['Gold', 'Medals'], aggfunc='first')

# 将多层列索引展平
df_pivot.columns = [f'{sport}_{metric}' for metric, sport in df_pivot.columns]

# 重置索引
df_pivot = df_pivot.reset_index()

# 打印处理后的数据前几行，确认转换是否正确
print("处理后的数据前几行：")
print(df_pivot.head())

# 将原始数据中的其他列合并到处理后的数据中
# 从原始数据中提取需要保留的列（包括 NOC、Year 和 isHost 列）
other_columns = df[['NOC', 'Year', 'isHost'] + df.columns[5:].tolist()]  # 包含 NOC、Year 和 isHost 列

# 将 other_columns 合并到 df_pivot 中
# 使用 merge 确保按照 NOC 和 Year 正确匹配
df_final = pd.merge(df_pivot, other_columns, on=['NOC', 'Year'], how='left')

# 去除空白行
df_final.dropna(how='all', inplace=True)

# 去除重复行
df_final.drop_duplicates(inplace=True)

# 将所有空值替换为 0
df_final = df_final.fillna(0)

# 保存结果
df_final.to_csv(r'.\data\Processed_host_countries_medals.csv', index=False)

print("处理完成，结果已保存到 .\data\Processed_host_countries_medals.csv")

原始数据前几行：
         NOC  Year      Sport  Gold  Medals  isHost  AthletesCount  Diving  \
0  Australia  1896  Athletics     2       2       0              5       0   
1  Australia  1896     Tennis     0       1       0              5       0   
2  Australia  1900  Athletics     1       4       0              6       0   
3  Australia  1900   Swimming     2       2       0              6       0   
4  Australia  1904   Swimming     0       4       0              6       2   

   Marathon Swimming  Swimming  ...  Beach  Indoor.1  Water Motorsports  \
0                  0         4  ...      0         0                  0   
1                  0         4  ...      0         0                  0   
2                  0         7  ...      0         0                  1   
3                  0         7  ...      0         0                  1   
4                  0         9  ...      0         0                  0   

   Weightlifting  Freestyle  Greco-Roman  Total events  Total disciplin

<>:43: SyntaxWarning: invalid escape sequence '\d'
<>:43: SyntaxWarning: invalid escape sequence '\d'
C:\Users\ROG\AppData\Local\Temp\ipykernel_39400\2862008123.py:43: SyntaxWarning: invalid escape sequence '\d'
  print("处理完成，结果已保存到 .\data\Processed_host_countries_medals.csv")
